In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
import tensorflow.contrib.eager as tfe

In [3]:
tf.enable_eager_execution()

#### Import model, training function 

In [4]:
from annsa.model_classes import (cnn1d_model_features,
                                 CNN1D,
                                 generate_random_cnn1d_architecture,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [5]:
dataset = np.load('../dataset_generation/testing_dataset_full_200keV_100.npy')

In [6]:
all_spectra = np.add(dataset.item()['sources'], dataset.item()['backgrounds'])
all_keys = dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys)

## Train network

### Define hyperparameters

In [7]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='CNN-kfoldsfull'

In [8]:
def generate_random_cnn1d_architecture(cnn_filters_choices,
                                       cnn_kernel_choices,
                                       pool_size_choices):
    """
    Generates a random 1d convolutional neural network based on a set of
    predefined architectures.

    Returns
    -------
    model_features : class
        Class that describes the structure of a 1D convolution neural network.

    """

    cnn_filters = choice(cnn_filters_choices)
    cnn_kernel_choice = choice(cnn_kernel_choices)
    pool_size_choice = choice(pool_size_choices)

    cnn_kernel = cnn_kernel_choice*(len(cnn_filters))
    cnn_strides = (1,)*(len(cnn_filters))
    pool_size = pool_size_choice*(len(cnn_filters))
    pool_strides = (2,)*(len(cnn_filters))

    number_layers = np.random.randint(1, 4)
    dense_nodes = (10**np.random.uniform(1,
                                         np.log10(1024/(2**len(
                                             cnn_filters))),
                                         number_layers)).astype('int')
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)

    model_features = cnn1d_model_features(
            trainable=None,
            learining_rate=None,
            batch_size=None,
            output_size=None,
            scaler=None,
            activation_function=None,
            output_function=None,
            Pooling=None,
            l2_regularization_scale=None,
            dropout_probability=None,
            cnn_filters=cnn_filters,
            cnn_kernel=cnn_kernel,
            cnn_strides=cnn_strides,
            pool_size=pool_size,
            pool_strides=pool_strides,
            dense_nodes=dense_nodes
            )

    return model_features

def make_model():
    '''
    Makes a random model given some parameters.

    '''
    cnn_filters_choices = ((4, 8),
                       (8, 16),
                       (16, 32),
                       (4,),
                       (8,),
                       (16,),
                       (32,),
                       (4, 8, 16),
                       (8, 16, 32),
                      )

    
    cnn_kernel_choices = ((2,), (4,), (8,), (16,))
    pool_size_choices = ((2,), (4,), (8,), (16,))

    
    model_features = generate_random_cnn1d_architecture(
        cnn_filters_choices= cnn_filters_choices,
        cnn_kernel_choices=cnn_kernel_choices,
        pool_size_choices=pool_size_choices,
    )
    model_features.trainable = True
    model_features.learining_rate = 10**np.random.uniform(-4,-1)
    model_features.batch_size = 2**np.random.randint(4,6)
    model_features.output_size = all_keys_binarized.shape[1]
    model_features.scaler = choice([make_pipeline(FunctionTransformer(np.log1p, validate=True)),
                                    make_pipeline(FunctionTransformer(np.sqrt, validate=True))])
    model_features.activation_function = tf.nn.relu
    model_features.output_function = None
    model_features.Pooling = tf.layers.MaxPooling1D
    model_features.l2_regularization_scale = 10**np.random.uniform(-3,0)
    model_features.dropout_probability = np.random.uniform(0,1)
    model_features.pool_strides = ((2,2,2))
    number_layers = choice([2])
    dense_nodes = 2**np.random.randint(4,8,number_layers)
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)
    
    model_features.dense_nodes = dense_nodes

    model = CNN1D(model_features)

    return model, model_features 

### Search hyperparameters

In [ ]:
def save_features(model_features,
                  model_id,
                  hyperparameter_index):
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index), 'wb+') as f:
        pickle.dump(model_features,f)
    return None

skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []

for network_id in range(number_hyperparameters_to_search):
    print(network_id)
    model, model_features = make_model()
    save_features(model_features,
                  model_id,
                  network_id)
    
    k_folds_errors = []
    for train_index, test_index in skf.split(all_spectra, all_keys):
        # reset model on each iteration
        model = CNN1D(model_features)
        optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_keys_binarized[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_keys_binarized[test_index],
                model=model,
                optimizer=optimizer,
                num_epochs=200,
                obj_cost=model.cross_entropy,
                earlystop_cost_fn=model.f1_error,
                earlystop_patience=10,
                not_learning_patience=10,
                not_learning_threshold=0.9,
                verbose=True,
                fit_batch_verbose=10,
                data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        all_kf_errors.append(earlystop_errors_tmp)

    testing_errors.append(np.average(k_folds_errors))
    np.save('./final-models/final_test_errors_'+model_id, testing_errors)
    np.save('./final-models/final_kf_errors_'+model_id, all_kf_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

0
Epoch 10: CostFunc loss: 0.00 11.58, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 16.39 Early stopfctn: 16.39
Epoch 10: CostFunc loss: 0.00 12.23, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 16.04 Early stopfctn: 16.04
Epoch 10: CostFunc loss: 0.00 10.26, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 15.47 Early stopfctn: 15.47
Epoch 10: CostFunc loss: 0.00 12.83, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 17.35 Early stopfctn: 17.35
Epoch 10: CostFunc loss: 0.00 10.72, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 16.05 Early stopfctn: 16.05
1
Epoch 10: CostFunc loss: 0.00 3.36, EarlyStop loss: 0.00 0.92
Test error at early stop: Objectives fctn: 18.51 Early stopfctn: 18.51
Epoch 10: CostFunc loss: 0.00 3.41, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 15.49 Early stopfctn: 15.49
Epoch 10: CostFunc loss: 0.00 3.24, EarlyStop loss: 0.00 0.9

Test error at early stop: Objectives fctn: 3.42 Early stopfctn: 3.42
Epoch 10: CostFunc loss: 0.00 3.54, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.14 Early stopfctn: 4.14
12
Epoch 10: CostFunc loss: 0.00 3.62, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.68 Early stopfctn: 4.68
Epoch 10: CostFunc loss: 0.00 3.66, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.87 Early stopfctn: 4.87
Epoch 10: CostFunc loss: 0.00 3.60, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.70 Early stopfctn: 4.70
Epoch 10: CostFunc loss: 0.00 3.67, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.95 Early stopfctn: 4.95
Epoch 10: CostFunc loss: 0.00 3.64, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.95 Early stopfctn: 4.95
13
Epoch 10: CostFunc loss: 0.00 3.72, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.12 Early stopfctn: 4.12
Epoch 10

Epoch 30: CostFunc loss: 0.00 1.02, EarlyStop loss: 0.00 0.22
Epoch 40: CostFunc loss: 0.00 0.85, EarlyStop loss: 0.00 0.16
Epoch 50: CostFunc loss: 0.00 0.85, EarlyStop loss: 0.00 0.17
Test error at early stop: Objectives fctn: 0.78 Early stopfctn: 0.78
20
Epoch 10: CostFunc loss: 0.00 3.70, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.86 Early stopfctn: 3.86
Epoch 10: CostFunc loss: 0.00 3.53, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.72 Early stopfctn: 3.72
Epoch 10: CostFunc loss: 0.00 3.71, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.91 Early stopfctn: 3.91
Epoch 10: CostFunc loss: 0.00 3.77, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.93 Early stopfctn: 3.93
Epoch 10: CostFunc loss: 0.00 3.57, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.76 Early stopfctn: 3.76
21
Epoch 10: CostFunc loss: 0.00 3.49, EarlyStop loss: 0.00 0.97
Test error at early st

Epoch 10: CostFunc loss: 0.00 3.52, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 5.01 Early stopfctn: 5.01
Epoch 10: CostFunc loss: 0.00 3.55, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 5.58 Early stopfctn: 5.58
Epoch 10: CostFunc loss: 0.00 3.46, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.66 Early stopfctn: 4.66
32
Epoch 10: CostFunc loss: 0.00 3.76, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 8.87 Early stopfctn: 8.87
Epoch 10: CostFunc loss: 0.00 3.43, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.38 Early stopfctn: 4.38
Epoch 10: CostFunc loss: 0.00 4.13, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 11.80 Early stopfctn: 11.80
Epoch 10: CostFunc loss: 0.00 3.73, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 7.66 Early stopfctn: 7.66
Epoch 10: CostFunc loss: 0.00 3.28, EarlyStop loss: 0.00 0.91
Test error at ea

Epoch 10: CostFunc loss: 0.00 8.65, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 18.02 Early stopfctn: 18.02
Epoch 10: CostFunc loss: 0.00 6.41, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 13.93 Early stopfctn: 13.93
Epoch 10: CostFunc loss: 0.00 7.66, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 16.35 Early stopfctn: 16.35
Epoch 10: CostFunc loss: 0.00 7.20, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 14.61 Early stopfctn: 14.61
Epoch 10: CostFunc loss: 0.00 7.22, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 15.52 Early stopfctn: 15.52
41
Epoch 10: CostFunc loss: 0.00 4.00, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.15 Early stopfctn: 4.15
Epoch 10: CostFunc loss: 0.00 4.05, EarlyStop loss: 0.00 0.93
Test error at early stop: Objectives fctn: 4.33 Early stopfctn: 4.33
Epoch 10: CostFunc loss: 0.00 3.82, EarlyStop loss: 0.00 0.92
Test err

Test error at early stop: Objectives fctn: 3.58 Early stopfctn: 3.58
51
Epoch 10: CostFunc loss: 0.00 6.58, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 14.52 Early stopfctn: 14.52
Epoch 10: CostFunc loss: 0.00 6.34, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 14.44 Early stopfctn: 14.44
Epoch 10: CostFunc loss: 0.00 6.42, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 15.11 Early stopfctn: 15.11
Epoch 10: CostFunc loss: 0.00 7.90, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 16.50 Early stopfctn: 16.50
Epoch 10: CostFunc loss: 0.00 6.05, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 13.56 Early stopfctn: 13.56
52
Epoch 10: CostFunc loss: 0.00 3.50, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.96 Early stopfctn: 3.96
Epoch 10: CostFunc loss: 0.00 3.60, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.11 Early stopfctn: 4.1

Epoch 10: CostFunc loss: 0.00 4.05, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 5.88 Early stopfctn: 5.88
Epoch 10: CostFunc loss: 0.00 4.20, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 6.58 Early stopfctn: 6.58
63
Epoch 10: CostFunc loss: 0.00 3.52, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.59 Early stopfctn: 3.59
Epoch 10: CostFunc loss: 0.00 3.49, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.55 Early stopfctn: 3.55
Epoch 10: CostFunc loss: 0.00 3.48, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.54 Early stopfctn: 3.54
Epoch 10: CostFunc loss: 0.00 3.49, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.57 Early stopfctn: 3.57
Epoch 10: CostFunc loss: 0.00 3.52, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.60 Early stopfctn: 3.60
64
Epoch 10: CostFunc loss: 0.00 3.98, EarlyStop loss: 0.00 0.97
Test error at e